In [ ]:
# pravimo boot-a
# 1 : pobedio
# 0 : nereseno
# -1 : izgubio
# iz ugla X maksimizujem, iz ugla O minimuzejemo
# gledamo sta najbolje protivnik moze da odigra
# skoro sve igre su zasonave na minmax-u  

In [ ]:
# stanje igre
class XOState:
  empty = ' '

  def __init__(self):
    self.board = [
        [XOState.empty, XOState.empty, XOState.empty],
        [XOState.empty, XOState.empty, XOState.empty],
        [XOState.empty, XOState.empty, XOState.empty]
    ]
    self.curr_player = 'X'
    self.move_count = 0;
    self.last_move = None   # vrsta i kolona dva int-a

  def play_move(self, move):
    i, j = move[0], move[1]
    
    # pre upisa X ili O treba da proverimo da vec nije upisano nesto
    self.board[i][j] = self.curr_player
    self.curr_player = 'X' if self.curr_player == 'O' else 'O'
    self.last_move = move
    self.move_count += 1

  def draw_board(self):
    print(" | ".join(self.board[0]))
    # print("-----------")
    print(" | ".join(self.board[1]))
    # print("-----------")
    print(" | ".join(self.board[2]))

def read_next_move_from_stdin():
  move = input().split(',')

  return [int(move[0]), int(move[1])]

# ocekujem kog je tipa, naveli smo tip kog ocekujemo
def end(current_state: XOState):
  winner = get_winner(current_state)

  return winner is not None or current_state.move_count == 9

def get_winner(current_state):
  board = current_state.board 

  # kolone
  for i in range(3):
    if board[0][i] != XOState.empty and board[0][i] == board[1][i] and board[0][1] == board[2][i]:
      return board[0][i]
  
  # vrste
  for  i in range(3):
    if board[i][0] != XOState.empty and board[i][0] == board[i][1] and board[i][0] == board[i][2]:
      return board[i][0]

  # dijagonale
  if (board[0][0] != XOState.empty and board[0][0] == board[1][1] and board[0][0] == board[2][2]):
    return board[0][0]

  # sporedna
  if (board[0][2] != XOState.empty and board[0][2] == board[1][1] and board[0][2] == board[0][1]):
    return board[0][2]

  return None

In [ ]:
# game = XOState()
# game.draw_board()

  |   |  
  |   |  
  |   |  


In [ ]:
# end(game)

False

In [ ]:
def evaluate(current_state: XOState):
  winner = get_winner(current_state)
  result = 0

  if winner == 'X':
    result = 1 / (current_state.move_count)
  elif winner == 'O':
    result = -1 / (current_state.move_count)
  
  return result

In [ ]:
import copy
def get_next_states(current_state: XOState):
  result = []

  for i in range(3):
    for j in range(3):
      if current_state.board[i][j] == XOState.empty:
        # duboko kopiranje - kopiram cele liste a ne reference
        next_state = copy.deepcopy(current_state)
        next_state.play_move([i, j])
        result.append(next_state)

  return result

In [ ]:
# next_states = get_next_states(game)

In [ ]:
# print(next_states[2].board)

[[' ', ' ', 'X'], [' ', ' ', ' '], [' ', ' ', ' ']]


In [ ]:
# def Max(current_state):
#   if end(current_state):
#     return evaluate(current_state), current_state

#   current_best_value = float('-inf')
#   # potomci - moguci potezi protivnika
#   best_move = None 

#   for next_state in get_next_states(current_state):
#     opponent_best_move, _ = Min(next_state)

#     if (opponent_best_move > current_best_value):
#       current_best_value = opponent_best_move
#       best_move = next_state 

#   return current_best_value, best_move

In [ ]:
from prompt_toolkit.application import current
def Max(current_state, alpha = float('-inf'), beta = float('inf')):
  if end(current_state):
    return evaluate(current_state), current_state

  current_best_value = float('-inf')
  # potomci - moguci potezi protivnika
  best_move = None 

  for next_state in get_next_states(current_state):
    opponent_best_move, _ = Min(next_state, alpha, beta)

    if (opponent_best_move > current_best_value):
      current_best_value = opponent_best_move
      best_move = next_state 

    if current_best_value >= beta:
      return current_best_value, next_state

    if current_best_value > alpha:
      alpha = current_best_value

  return current_best_value, best_move

In [ ]:
# def Min(current_state):
#   if end(current_state):
#     return evaluate(current_state), current_state 

#   current_best_value = float('inf')
#   best_move = None

#   for next_state in get_next_states(current_state):
#     opponent_best_move, _ = Max(next_state)

#     # current_best_move_value ???
#     if opponent_best_move < current_best_value:
#       current_best_value = opponent_best_move
#       best_move = next_state 

#   return current_best_value, best_move

In [ ]:
def Min(current_state, alpha = float('-inf'), beta = float('inf')):
  if end(current_state):
    return evaluate(current_state), current_state 

  current_best_value = float('inf')
  best_move = None

  for next_state in get_next_states(current_state):
    opponent_best_move, _ = Max(next_state, alpha, beta)

    # current_best_move_value ???
    if opponent_best_move < current_best_value:
      current_best_value = opponent_best_move
      best_move = next_state 

    if current_best_value <= alpha:
      return current_best_value, next_state

    if current_best_value < beta:
      beta = current_best_value = beta

  return current_best_value, best_move

In [ ]:
def get_next_computer_move(game, func):
  # ne treba mi vrednost vec samo state, koje mi je najbolje stanje
  _, state = func(game)

  return state.last_move

In [ ]:
game = XOState()
game.draw_board()

while True:
  next_move = read_next_move_from_stdin()
  game.play_move(next_move)
  game.draw_board()

  if (get_winner(game) == 'X'):
    print("X won!")
    break
  if end(game):
    print('Tie')
    break

  next_move = get_next_computer_move(game, Min)
  game.play_move(next_move)
  game.draw_board()

  if (get_winner(game) == 'O'):
    print('player O won!')
    break

  |   |  
  |   |  
  |   |  
2,2
  |   |  
  |   |  
  |   | X


AttributeError: ignored

In [ ]:
# alpha-beta odsecanje